<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>1D ConvNet on Sentiment Analysis</H1></u></center>

# First run this from terminal

In [78]:
# import nltk
# nltk.download('stopwords')

In [79]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
import plotly.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

In [80]:
#import time
#start_time = time.time()
import pandas as pd
import numpy as np

In [81]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)
set_session(sess)

In [82]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, Embedding, LSTM, CuDNNLSTM, Dropout, Flatten, Input
from keras.datasets import imdb

In [83]:
num_words = 50000

In [84]:
# def load_data(path='imdb.npz', num_words=None, skip_top=0,
#               maxlen=None, seed=113,
#               start_char=1, oov_char=2, index_from=3, **kwargs):

#     with np.load(path) as f:
#         x_train, labels_train = f['x_train'], f['y_train']
#         x_test, labels_test = f['x_test'], f['y_test']

#     np.random.seed(seed)
#     indices = np.arange(len(x_train))
#     np.random.shuffle(indices)
#     x_train = x_train[indices]
#     labels_train = labels_train[indices]

#     indices = np.arange(len(x_test))
#     np.random.shuffle(indices)
#     x_test = x_test[indices]
#     labels_test = labels_test[indices]

#     xs = np.concatenate([x_train, x_test])
#     labels = np.concatenate([labels_train, labels_test])

#     if start_char is not None:
#         xs = [[start_char] + [w + index_from for w in x] for x in xs]
#     elif index_from:
#         xs = [[w + index_from for w in x] for x in xs]

#     if maxlen:
#         xs, labels = _remove_long_seq(maxlen, xs, labels)
#         if not xs:
#             raise ValueError('After filtering for sequences shorter than maxlen=' +
#                              str(maxlen) + ', no sequence was kept. '
#                              'Increase maxlen.')
#     if not num_words:
#         num_words = max([max(x) for x in xs])

#     if oov_char is not None:
#         xs = [[w if (skip_top <= w < num_words) else oov_char for w in x]
#               for x in xs]
#     else:
#         xs = [[w for w in x if skip_top <= w < num_words]
#               for x in xs]

#     idx = len(x_train)
#     x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
#     x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])

#     return (x_train, y_train), (x_test, y_test)

In [85]:
#(train_data, train_labels), (test_data, test_labels) = load_data(path='imdb.npz',num_words=10000)

In [86]:
# (X_train, y_train), (X_test, y_test) = load_data(path='imdb.npz',num_words=10000)

In [87]:
df = pd.read_csv('consumer_complaints_small.csv')

In [88]:
df.columns

Index(['product', 'consumer_complaint_narrative'], dtype='object')

In [89]:
# X = 'consumer_complaint_narrative', Y = 'product'

df = df[['consumer_complaint_narrative','product']]

In [90]:
df.head()

,consumer_complaint_narrative,product
0,XXXX has claimed I owe them {$27.00} for XXXX ...,Debt collection
1,Due to inconsistencies in the amount owed that...,Consumer Loan
2,In XX/XX/XXXX my wages that I earned at my job...,Mortgage
3,I have an open and current mortgage with Chase...,Mortgage
4,XXXX was submitted XX/XX/XXXX. At the time I s...,Mortgage


In [91]:
len(df)

66806

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66806 entries, 0 to 66805
Data columns (total 2 columns):
consumer_complaint_narrative    66806 non-null object
product                         66806 non-null object
dtypes: object(2)
memory usage: 1.0+ MB


In [93]:
df['product'].value_counts()

Debt collection            17552
Mortgage                   14919
Credit reporting           12526
Credit card                 7929
Bank account or service     5711
Consumer Loan               3678
Student loan                2128
Prepaid card                 861
Payday loan                  726
Money transfers              666
Other financial service      110
Name: product, dtype: int64

In [94]:
df['product'].value_counts().sort_values(ascending=False).iplot(kind='bar', yTitle='Number of Reviews', 
                                                                title='Number reviews in each rating')

## Down-sampling the data

In [95]:
# # Using Ramdom Under Sampling (RUS) technique
# X = df.iloc[:, :1].values
# y = df.loc[:, 'Score'].values

# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(return_indices=True)
# X_rus, y_rus, id_rus = rus.fit_sample(X, y)

In [96]:
# X_rus[:,0]

In [97]:
#y_rus[:,1]

In [98]:
# X_rus.shape

In [99]:
# X_rus[10]

In [100]:
# len(X)

In [101]:
# len(X_rus)

In [102]:
# id_rus

In [103]:
# X_rus = X_rus[:,0]

In [104]:
# X_rus.shape

In [105]:
# y_rus

In [106]:
# y_rus.shape

In [107]:
##y_rus = y_rus[:,0]

In [108]:
# df = pd.DataFrame({'Text':X_rus,'Score':y_rus})

## df = pd.DataFrame({'x':x, 'y':y})

In [109]:
# df['Text'][1]

In [110]:
def print_plot(index):
    example = df[df.index == index][['consumer_complaint_narrative', 'product']].values[0]
    if len(example) > 0:
        print(example[0])
        print('product:', example[1])

In [111]:
print_plot(10)

I am very disappointed that the CFPB did not help to resolve this fraudulant loan. # XXXX between XXXX XXXX and XXXX, FF ( AKA ) One West. Case # XXXX with CFPB. 

product: Mortgage


In [112]:
print_plot(100)

I contacted Drivetime on several different occasions regarding a car loan I had with them account # XXXX i do not have the number for the other loan that was removed also from the credit bureau with a positive credit rating that was closed and paid in full. The company insisted on avoiding an agreement entered into by myself and XXXX of their company representative in a corporate position they have the agreed upon resolution and can provide your agency with a copy of the same because i do not have the capability to do so with my computer. Drivetime during the month of XXXX XXXX removed my credit reporting information from XXXX credit bureaus reporting agencies I believe XXXX and XXXX XXXX I discovered in or about XXXX XXXX that my positive credit ratings were no longer in the XXXX and XXXX credit bureaus and that another positive account for another car loan was no longer reporting in my credit file. these removals adversely affecxted my credit ratings and as such my ability to purchas

In [113]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].apply(clean_text)

In [114]:
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].str.replace('\d+', '')

In [115]:
print_plot(10)

disappointed cfpb help resolve fraudulant loan # ff aka one west case # cfpb
product: Mortgage


In [116]:
print_plot(100)

contacted drivetime several different occasions regarding car loan account # number loan removed also credit bureau positive credit rating closed paid full company insisted avoiding agreement entered company representative corporate position agreed upon resolution provide agency copy capability computer drivetime month removed credit reporting information credit bureaus reporting agencies believe discovered positive credit ratings longer credit bureaus another positive account another car loan longer reporting credit file removals adversely affected credit ratings ability purchase something lower interest rate never missed payment drivetime company car loans inclusive different auto loans destroy credit ratings despicable spoke today regarding issues response drivetime process updating consumers credit files consumers credit files removed credit bureaus updated however know process performed records updated corrected process timeframes process take furthermore process involves consumer

# ==================================

## Test- only first 5000 observations

In [117]:
len(df)

66806

In [118]:
df.head()

,consumer_complaint_narrative,product
0,claimed owe years despite proof payment sent ...,Debt collection
1,due inconsistencies amount owed told bank amou...,Consumer Loan
2,wages earned job decreased almost half knew tr...,Mortgage
3,open current mortgage chase bank # chase repor...,Mortgage
4,submitted time submitted complaint dealt rushm...,Mortgage


# Shuffle the dataframe index, inplace

In [119]:
# df = df.sample(frac=1).reset_index(drop=True)

In [120]:
#df = df[:5000]

In [121]:
df.head()

,consumer_complaint_narrative,product
0,claimed owe years despite proof payment sent ...,Debt collection
1,due inconsistencies amount owed told bank amou...,Consumer Loan
2,wages earned job decreased almost half knew tr...,Mortgage
3,open current mortgage chase bank # chase repor...,Mortgage
4,submitted time submitted complaint dealt rushm...,Mortgage


In [122]:
df['product'].value_counts()

Debt collection            17552
Mortgage                   14919
Credit reporting           12526
Credit card                 7929
Bank account or service     5711
Consumer Loan               3678
Student loan                2128
Prepaid card                 861
Payday loan                  726
Money transfers              666
Other financial service      110
Name: product, dtype: int64

# ====================================

In [123]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['consumer_complaint_narrative'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 56802 unique tokens.


In [124]:
X = tokenizer.texts_to_sequences(df['consumer_complaint_narrative'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (66806, 250)


In [125]:
Y = pd.get_dummies(df['product']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (66806, 11)


In [126]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(53444, 250) (53444, 11)
(13362, 250) (13362, 11)


# finished data preprocessing

# ====================================================================

# start building CNN 

In [127]:
print(len(X_train), 'train_data')
print(len(X_test), 'test_data')

53444 train_data
13362 test_data


In [128]:
#(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

In [129]:
max_len = 256
embedding_size = 50
batch_size = 128

## Sequences per time

In [130]:
print(X_train[2])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     1    14     2   313    47    20    35    48  3114   313
    47    10     2    13     1   221    45     5    35    48   868    48
   649     2   601  1012    20    33    43  1428    33    65     5   574
   313    47   590     5   320     5   110   100     6     5   274  1301
  2612   282   624   561     1    10   153   313    47   601    10  1012
    43   274     5    44   216   195   601    52     2   759    48  1012
    20    16   820   227  3225   590   530  1013   

In [131]:
len(X_train[1])

250

In [132]:
pad  =  'post' #'pre'
#Convert our lists to equal length sequences
X_train_pad = pad_sequences(X_train, maxlen=max_len, padding=pad, truncating=pad)
X_test_pad = pad_sequences(X_test, maxlen=max_len, padding=pad, truncating=pad)

In [133]:
X_train_pad.shape

(53444, 256)

In [134]:
X_train_pad[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [135]:
X_train_pad[4499]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [136]:
#print("--- %s seconds ---" % (time.time() - start_time))

## Creating the model (Model 0):

In [137]:
model0 = Sequential()

In [138]:
model0.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_len,
                    name='layer_embedding'))

In [139]:
model0.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))

In [140]:
model0.add(MaxPooling1D(pool_size=2))

In [141]:
model0.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))

In [142]:
model0.add(MaxPooling1D(pool_size=2))

In [143]:
model0.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))

In [144]:
model0.add(MaxPooling1D(pool_size=2))

In [145]:
model0.add(Dropout(0.5))

In [146]:
model0.add(Flatten())

In [147]:
model0.add(Dense(250, activation='relu'))

In [148]:
#model0.add(Dense(5, activation='softmax', name='classification'))

model0.add(Dense(11, activation='softmax'))

## Compiling the model:

In [149]:
model0.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [150]:
model0.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 256, 50)           2500000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 256, 128)          19328     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 128, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 128, 128)          49280     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 64, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 64, 128)           49280     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 32, 128)           0         
__________

## Training

In [151]:
#%%time
model0.fit(X_train_pad, Y_train, validation_split=0.05, epochs=10, batch_size=batch_size)

Train on 50771 samples, validate on 2673 samples
Epoch 1/10
50771/50771 [==============================] - 108s 2ms/step - loss: 0.1437 - acc: 0.9492 - val_loss: 0.1074 - val_acc: 0.9636
Epoch 2/10
50771/50771 [==============================] - 107s 2ms/step - loss: 0.0909 - acc: 0.9686 - val_loss: 0.0866 - val_acc: 0.9698
Epoch 3/10
50771/50771 [==============================] - 107s 2ms/step - loss: 0.0764 - acc: 0.9743 - val_loss: 0.0813 - val_acc: 0.9721
Epoch 4/10
50771/50771 [==============================] - 106s 2ms/step - loss: 0.0674 - acc: 0.9772 - val_loss: 0.0866 - val_acc: 0.9683
Epoch 5/10
50771/50771 [==============================] - 105s 2ms/step - loss: 0.0604 - acc: 0.9798 - val_loss: 0.0833 - val_acc: 0.9721
Epoch 6/10
50771/50771 [==============================] - 105s 2ms/step - loss: 0.0545 - acc: 0.9818 - val_loss: 0.0829 - val_acc: 0.9727
Epoch 7/10
50771/50771 [==============================] - 106s 2ms/step - loss: 0.0483 - acc: 0.9839 - val_loss: 0.0932 - v

## Testing

In [152]:
#%%time
eval_ = model0.evaluate(X_test_pad, Y_test)

13362/13362 [==============================] - 9s 686us/step


In [153]:
print("Loss: {0:.5}".format(eval_[0]))
print("Accuracy: {0:.2%}".format(eval_[1]))

Loss: 0.12273
Accuracy: 96.81%


# ===============================================

# Now show the F-1 score and Accuracy

In [154]:
accr = model0.evaluate(X_test_pad,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))



13362/13362 [==============================] - 9s 676us/step
Test set
  Loss: 0.123
  Accuracy: 0.968


In [155]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [156]:
def decode(datum):
    return np.argmax(datum)

In [157]:
# for i in range(Y_test.shape[0]):
#     datum = Y_test[i]
#     print('index: %d' % i)
#     print('encoded datum: %s' % datum)
#     decoded_datum = decode(Y_test[i])
#     print('decoded datum: %s' % decoded_datum)
#     print()

In [158]:
Y_test = np.array([decode(i) for i in Y_test])

In [159]:
yhat_probs = model0.predict(X_test_pad, verbose=0)
# predict crisp classes for test set
yhat_classes = model0.predict_classes(X_test_pad, verbose=0)

In [160]:
yhat_classes

array([4, 6, 2, ..., 4, 3, 3])

In [161]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Y_test, yhat_classes)
print('Accuracy: %f' % accuracy)

Accuracy: 0.817467


In [162]:
f1_score(Y_test, yhat_classes, average='macro') 

/usr/local/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



0.6695491035548081

In [163]:
f1_score(Y_test, yhat_classes, average='micro') 

0.8174674449932644

In [164]:
f1_score(Y_test, yhat_classes, average=None) 

/usr/local/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



array([0.77889228, 0.66503667, 0.80048295, 0.84819553, 0.7962178 ,
       0.63709677, 0.9141398 , 0.        , 0.39534884, 0.71473354,
       0.81489595])

In [165]:
f1_score(Y_test, yhat_classes, average='weighted') 

/usr/local/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



0.8155272907373908

In [166]:
unique_elements, counts_elements = np.unique(Y_test, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[   0    1    2    3    4    5    6    7    8    9   10]
 [1155  746 1615 2502 3496  150 2941   21  153  168  415]]


In [167]:
unique_elements, counts_elements = np.unique(yhat_classes, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[   0    1    2    3    4    5    6    8    9   10]
 [1138  890 1698 2735 2638   98 3325  191  151  498]]


In [168]:
# confusion matrix
matrix = confusion_matrix(Y_test, yhat_classes)
print(matrix)

[[ 893   18   96   25   11    9   72    0    4   23    4]
 [  17  544   37   37   25    0   52    0   23    0   11]
 [  75   50 1326   79   39    1   32    0    2    9    2]
 [  14   49   47 2221   76    0   81    0    4    0   10]
 [  36  162  153  342 2442    1  182    0   83    0   95]
 [  38    7    8    0    2   79    9    0    2    5    0]
 [  21   19    7   22    6    0 2864    0    1    0    1]
 [   5    3    2    1    2    3    3    0    1    0    1]
 [   4   33    2    3   22    3   16    0   68    0    2]
 [  33    0   18    1    1    1    0    0    0  114    0]
 [   2    5    2    4   12    1   14    0    3    0  372]]


# ==================================

## Model 1:

In [90]:
model1 = Sequential()

In [91]:
model1.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_len,
                    name='layer_embedding'))

In [92]:
model1.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))

In [93]:
model1.add(MaxPooling1D(pool_size=3))

In [94]:
model1.add(CuDNNLSTM(128))

In [95]:
model1.add(Dropout(0.5))

In [96]:
model1.add(Dense(1, activation='sigmoid', name='classification'))

## Compiling the model:

In [97]:
model1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [42]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 256, 50)           1000000   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 256, 128)          32128     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 85, 128)           0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 128)               132096    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
classification (Dense)       (None, 1)                 129       
Total params: 1,164,353
Trainable params: 1,164,353
Non-trainable params: 0
_________________________________________________________________


## Training the model:

In [43]:
%%time
model1.fit(X_train_pad, y_train, validation_split=0.05, epochs=10, batch_size=batch_size)

Train on 23750 samples, validate on 1250 samples
Epoch 1/10


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' with these attrs.  Registered devices: [CPU], Registered kernels:
  <no registered kernels>

	 [[{{node cu_dnnlstm_1/CudnnRNN}} = CudnnRNN[T=DT_FLOAT, direction="unidirectional", dropout=0, input_mode="linear_input", is_training=true, rnn_mode="lstm", seed=87654321, seed2=0](cu_dnnlstm_1/transpose, cu_dnnlstm_1/ExpandDims_1, cu_dnnlstm_1/ExpandDims_2, cu_dnnlstm_1/concat_1)]]

Caused by op 'cu_dnnlstm_1/CudnnRNN', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/li

## Testing the model:

In [44]:
%%time
eval_ = model1.evaluate(X_test_pad, y_test)

InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' with these attrs.  Registered devices: [CPU], Registered kernels:
  <no registered kernels>

	 [[{{node cu_dnnlstm_1/CudnnRNN}} = CudnnRNN[T=DT_FLOAT, direction="unidirectional", dropout=0, input_mode="linear_input", is_training=true, rnn_mode="lstm", seed=87654321, seed2=0](cu_dnnlstm_1/transpose, cu_dnnlstm_1/ExpandDims_1, cu_dnnlstm_1/ExpandDims_2, cu_dnnlstm_1/concat_1)]]

Caused by op 'cu_dnnlstm_1/CudnnRNN', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/li

In [45]:
print("Loss: {0:.5}".format(eval_[0]))
print("Accuracy: {0:.2%}".format(eval_[1]))

Loss: 6.958
Accuracy: 56.60%


## Conv1D model with multiple kernel sizes:

In [46]:
from keras.layers import concatenate

In [47]:
conv = []
filter_sizes = [3,4,5]

In [48]:
embedding_layer = Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_len,
                    name='layer_embedding')

In [49]:
seq_input = Input(shape=(max_len,), dtype='int32')
embed_seq = embedding_layer(seq_input)

for f in filter_sizes:
    _conv = Conv1D(filters=128,kernel_size=f,activation='relu')(embed_seq)
    _pool = MaxPooling1D(5)(_conv)
    conv.append(_pool)

In [50]:
_concat = concatenate(conv, axis=1)
_conv1= Conv1D(128, 5, activation='relu')(_concat)
_pool1 = MaxPooling1D(5)(_conv1)
_pool1 = Dropout(0.5)(_pool1)
_conv2 = Conv1D(128, 5, activation='relu')(_pool1)
_pool2 = MaxPooling1D(5)(_conv2)
_flat = Flatten()(_pool2)
_flat = Dropout(0.5)(_flat)
_dense = Dense(128, activation='relu')(_flat)
preds = Dense(1, activation='sigmoid')(_dense)

In [51]:
model = Model(seq_input, preds)

In [52]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
layer_embedding (Embedding)     (None, 256, 50)      1000000     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 254, 128)     19328       layer_embedding[0][0]            
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 253, 128)     25728       layer_embedding[0][0]            
__________________________________________________________________________________________________
conv1d_7 (

In [53]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [54]:
%%time
model.fit(X_train_pad, y_train, validation_split=0.05, epochs=10, batch_size=batch_size)

Train on 23750 samples, validate on 1250 samples
Epoch 1/10


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' with these attrs.  Registered devices: [CPU], Registered kernels:
  <no registered kernels>

	 [[{{node cu_dnnlstm_1/CudnnRNN}} = CudnnRNN[T=DT_FLOAT, direction="unidirectional", dropout=0, input_mode="linear_input", is_training=true, rnn_mode="lstm", seed=87654321, seed2=0](cu_dnnlstm_1/transpose, cu_dnnlstm_1/ExpandDims_1, cu_dnnlstm_1/ExpandDims_2, cu_dnnlstm_1/concat_1)]]

Caused by op 'cu_dnnlstm_1/CudnnRNN', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/li

In [55]:
%%time
eval_ = model.evaluate(X_test_pad, y_test)

InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' with these attrs.  Registered devices: [CPU], Registered kernels:
  <no registered kernels>

	 [[{{node cu_dnnlstm_1/CudnnRNN}} = CudnnRNN[T=DT_FLOAT, direction="unidirectional", dropout=0, input_mode="linear_input", is_training=true, rnn_mode="lstm", seed=87654321, seed2=0](cu_dnnlstm_1/transpose, cu_dnnlstm_1/ExpandDims_1, cu_dnnlstm_1/ExpandDims_2, cu_dnnlstm_1/concat_1)]]

Caused by op 'cu_dnnlstm_1/CudnnRNN', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/li

In [56]:
print("Loss: {0:.5}".format(eval_[0]))
print("Accuracy: {0:.2%}".format(eval_[1]))

Loss: 6.958
Accuracy: 56.60%


In [ ]:
#print("Total time used....")
#print("--- %s seconds ---" % (time.time() - start_time))

## Reference:

https://keras.io/layers/convolutional/